In [5]:
import cv2
import numpy as np

def segment_brain_tumor_opencv(image_path):
    """
    Performs a basic conceptual brain tumor segmentation using OpenCV.

    Args:
        image_path (str): Path to the brain image (e.g., an MRI slice).

    Returns:
        tuple: (original_image, segmented_output_image)
               Returns (None, None) if the image cannot be loaded.
               The segmented_output_image will have potential tumor regions contoured.
    """
    # 1. Load Image
    original_image = cv2.imread(image_path)
    if original_image is None:
        print(f"Error: Could not load image from {image_path}")
        return None, None

    # 2. Preprocessing
    # Convert to Grayscale
    gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur for noise reduction
    # Kernel size (e.g., (5,5)) can be tuned. SigmaX is set to 0 to auto-calculate.
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # 3. Thresholding to create a binary image
    # Otsu's thresholding automatically determines the optimal threshold value
    # from the image histogram. This is effective for bimodal histograms.
    # The choice of THRESH_BINARY or THRESH_BINARY_INV depends on whether the
    # tumor region is expected to be brighter or darker than the background.
    # Assuming tumor is brighter:
    _, thresholded_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # If the tumor is darker, you might try:
    # _, thresholded_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # 4. Morphological Operations to refine the binary mask
    # These operations help to remove noise and close gaps in the segmented regions.
    # Kernel for morphological operations
    kernel = np.ones((5, 5), np.uint8)

    # Opening: Erosion followed by Dilation. Useful for removing small noise spots (salt noise).
    opened_image = cv2.morphologyEx(thresholded_image, cv2.MORPH_OPEN, kernel, iterations=2)

    # Closing: Dilation followed by Erosion. Useful for filling small holes within foreground objects (pepper noise).
    # Depending on the image, you might want to apply closing as well, or instead.
    # For this example, we'll use the result of opening.
    # closed_image = cv2.morphologyEx(opened_image, cv2.MORPH_CLOSE, kernel, iterations=2)
    # processed_mask = closed_image
    processed_mask = opened_image


    # 5. Find Contours
    # Contours are the boundaries of connected components in the binary image.
    # cv2.RETR_EXTERNAL: retrieves only the extreme outer contours.
    # cv2.CHAIN_APPROX_SIMPLE: compresses horizontal, vertical, and diagonal segments, leaving only their end points.
    contours, hierarchy = cv2.findContours(processed_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 6. Filter and Draw Contours
    # In a real application, contours would be filtered based on size, shape, location,
    # intensity, texture, etc., often using machine learning.
    # Here, we'll draw contours that are above a certain area to filter out very small noise.
    segmented_output_image = original_image.copy()
    min_contour_area = 100 # This threshold is arbitrary and needs tuning.

    if contours:
        # You could sort contours by area if desired:
        # contours = sorted(contours, key=cv2.contourArea, reverse=True)
        
        potential_tumor_count = 0
        for i, c in enumerate(contours):
            area = cv2.contourArea(c)
            if area > min_contour_area:
                # Draw the contour on the original image
                cv2.drawContours(segmented_output_image, [c], -1, (0, 255, 0), 2) # Green color, 2px thickness
                potential_tumor_count += 1
        
        if potential_tumor_count == 0:
            print("No significant contours found based on the current parameters.")
    else:
        print("No contours found in the image.")

   

    return original_image, segmented_output_image, gray_image, blurred_image, thresholded_image, processed_mask


if __name__ == "__main__":
    # IMPORTANT: Replace 'path_to_your_brain_image.jpg' with the actual path to your image file.
    # This example needs an image where the tumor has a distinguishable intensity.
    # Common formats: .jpg, .png, .tif. For medical images like DICOM, you'd need a DICOM reader (e.g., pydicom).
    
    image_file_path = 'bt1.jpg' # <--- CHANGE THIS TO YOUR IMAGE PATH

    # As a placeholder, if you don't have an image, this script won't run correctly.
    # You can test with a publicly available brain MRI slice image that shows a tumor.
    # For example, search for "Brats dataset sample image".

    print(f"Attempting to process image: {image_file_path}")
    print("If 'path_to_your_brain_image.jpg' is not replaced, this will likely fail or use a non-existent file.")

    original, segmented, gray, blurred, thresh, morph = segment_brain_tumor_opencv(image_file_path)

    if original is not None and segmented is not None:
        print("Processing complete.")
        
        # Displaying images using OpenCV's imshow
        cv2.imshow("1. Original Image", original)
        cv2.imshow("2. Grayscale", gray)
        cv2.imshow("3. Blurred", blurred)
        cv2.imshow("4. Otsu Thresholded", thresh)
        cv2.imshow("5. Morphological Processed", morph)
        cv2.imshow("6. Conceptual Segmentation", segmented)

        print("Press any key to close the image windows.")
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Optionally, save the output
        # output_filename = "segmented_brain_tumor_output.png"
        # cv2.imwrite(output_filename, segmented)
        # print(f"Segmented image saved as {output_filename}")
    else:
        print(f"Image processing failed. Please check the image path and integrity: {image_file_path}")

Attempting to process image: bt1.jpg
If 'path_to_your_brain_image.jpg' is not replaced, this will likely fail or use a non-existent file.
Processing complete.
Press any key to close the image windows.
